In [6]:
import pandas as pd
df_base = pd.read_csv('data/lending-club-loan-data/loan.csv')

C:\Users\tkim25\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_base.shape

(887379, 74)

In [8]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 74 columns):
id                             887379 non-null int64
member_id                      887379 non-null int64
loan_amnt                      887379 non-null float64
funded_amnt                    887379 non-null float64
funded_amnt_inv                887379 non-null float64
term                           887379 non-null object
int_rate                       887379 non-null float64
installment                    887379 non-null float64
grade                          887379 non-null object
sub_grade                      887379 non-null object
emp_title                      835917 non-null object
emp_length                     842554 non-null object
home_ownership                 887379 non-null object
annual_inc                     887375 non-null float64
verification_status            887379 non-null object
issue_d                        887379 non-null object
loan_status          

In [9]:
df_filtered = df_base[["loan_amnt", "loan_status","grade", "int_rate", "term"]]

In [10]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887379 entries, 0 to 887378
Data columns (total 5 columns):
loan_amnt      887379 non-null float64
loan_status    887379 non-null object
grade          887379 non-null object
int_rate       887379 non-null float64
term           887379 non-null object
dtypes: float64(2), object(3)
memory usage: 33.9+ MB


- '36개월 대출'과 '60개월 대출'의 대출 총액 파악

In [12]:
df_filtered.head()

,loan_amnt,loan_status,grade,int_rate,term
0,5000.0,Fully Paid,B,10.65,36 months
1,2500.0,Charged Off,C,15.27,60 months
2,2400.0,Fully Paid,C,15.96,36 months
3,10000.0,Fully Paid,C,13.49,36 months
4,3000.0,Current,B,12.69,60 months


In [25]:
df_filtered['term'] = df_filtered.term.str.strip()

C:\Users\tkim25\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
term_to_loan_amnt_dict = {}

uniq_terms = df_filtered["term"].unique()
uniq_terms

array(['36 months', '60 months'], dtype=object)

In [27]:
for term in uniq_terms:
    loan_amnt_sum = df_filtered.loc[df_filtered["term"] == term, "loan_amnt"].sum()
    term_to_loan_amnt_dict[term] = loan_amnt_sum

In [28]:
term_to_loan_amnt = pd.Series(term_to_loan_amnt_dict)

In [29]:
term_to_loan_amnt

36 months    7.752507e+09
60 months    5.341005e+09
dtype: float64

- 각 대출 상태(불량/우량)에 따른 대출 등급 분포 파악

In [39]:
df_filtered['loan_status'] = df_filtered['loan_status'].str.strip()

C:\Users\tkim25\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
arr_total_status_category = df_filtered["loan_status"].unique()

In [41]:
arr_total_status_category

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

In [44]:
arr_bad_status_category = arr_total_status_category[[1, 3, 4, 5, 6, 8]]
arr_bad_status_category

array(['Charged Off', 'Default', 'Late (31-120 days)', 'In Grace Period',
       'Late (16-30 days)',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [45]:
df_filtered["bad_loan_status"] = df_filtered["loan_status"].isin(arr_bad_status_category)

C:\Users\tkim25\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_filtered.head()

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_status
0,5000.0,Fully Paid,B,10.65,36 months,False
1,2500.0,Charged Off,C,15.27,60 months,True
2,2400.0,Fully Paid,C,15.96,36 months,False
3,10000.0,Fully Paid,C,13.49,36 months,False
4,3000.0,Current,B,12.69,60 months,False


In [48]:
bad_loan_status_to_grades = df_filtered.loc[df_filtered["bad_loan_status"] == True, "grade"].value_counts()
bad_loan_status_to_grades

C    19054
D    15859
B    13456
E     9745
F     4383
A     3663
G     1269
Name: grade, dtype: int64

In [49]:
bad_loan_status_to_grades.sort_index()

A     3663
B    13456
C    19054
D    15859
E     9745
F     4383
G     1269
Name: grade, dtype: int64

- 대출 총액과 대출 이자율 간의 상관관계 파악

In [50]:
df_filtered["loan_amnt"].corr(df_filtered["int_rate"])

0.1450230992988384

## 파일 쓰기

- 여러분이 얻을 bad_loan_status_to_grades Series를 외부 파일로 쓰는 작업을 진행해보도록 합시다. 이는 DataFrame에 대해서도 동일하게 적용됩니다.


- Series.to_csv() 혹은 df.to_csv() 함수를 사용하면, 현재 Series 혹은 DataFrame 형태로 들고 있는 분석 결과물을 외부 파일로 손쉽게 쓸 수 있습니다. 해당 함수를 호출할 때, 저장할 파일의 경로와 구분자 등을 인자로 같이 입력합니다.

In [51]:
bad_loan_status_to_grades.to_csv("data/lending-club-loan-data/bad_loan_status.csv", sep=",")